# BYOL Model

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
!pip install byol-pytorch torchvision matplotlib scikit-learn

In [ ]:
# Define the source directory for downloaded zip files
source_dir="data"  # Root directory where datasets are stored
destination_dir="crops_for_byol"  # Destination directory to store the crops

# Step 0: Delete all existing files in the source directory using shell command
!rm -rf {source_dir}/*

# Step 1: Download the datasets using unique output file names
!wget -O d1.zip "https://baylor.box.com/shared/static/4fxus7eaalj12fpl7sv79qfiemobx2ea.zip"  # yolo9_cs_24_07_16
!wget -O d3.zip "https://baylor.box.com/shared/static/p8h57qch6qwbsqple1k4ttkiwd8qteci.zip"  # yolo9_hw_24_07_16
!wget -O d4.zip "https://baylor.box.com/shared/static/2sr9ceoiy2ap8rzi1njhaejqwdulehwe.zip"  # yolo9_sr_24_07_16
!wget -O d5.zip "https://baylor.box.com/shared/static/ste8zq1un7crpelluqlh48nj4xsepjqk.zip"  # yolo9_tf_24_07_16



--2025-04-26 20:32:41--  https://baylor.box.com/shared/static/4fxus7eaalj12fpl7sv79qfiemobx2ea.zip
Resolving baylor.box.com (baylor.box.com)... 74.112.186.157, 2620:117:bff0:12d::
Connecting to baylor.box.com (baylor.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/4fxus7eaalj12fpl7sv79qfiemobx2ea.zip [following]
--2025-04-26 20:32:41--  https://baylor.box.com/public/static/4fxus7eaalj12fpl7sv79qfiemobx2ea.zip
Reusing existing connection to baylor.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://baylor.app.box.com/public/static/4fxus7eaalj12fpl7sv79qfiemobx2ea.zip [following]
--2025-04-26 20:32:42--  https://baylor.app.box.com/public/static/4fxus7eaalj12fpl7sv79qfiemobx2ea.zip
Resolving baylor.app.box.com (baylor.app.box.com)... 74.112.186.157, 2620:117:bff0:12d::
Connecting to baylor.app.box.com (baylor.app.box.com)|74.112.186.157|:443... connected.
HTTP request

In [ ]:
import subprocess
import pathlib
import time

# Define the source directory for downloaded zip files
source_dir = "data"  # Root directory where datasets are stored

# List of all downloaded zip files (use the names that you assigned in the wget step)
zip_files = ["d1.zip", "d2.zip", "d3.zip", "d4.zip", "d5.zip", "d6.zip",
             "d7.zip", "d8.zip", "d9.zip", "d10.zip", "d11.zip", "d12.zip"]

# Step 1: Unzip all the downloaded datasets
for zip_file in zip_files:
    unzip_command = ["unzip", "-q", zip_file, "-d", source_dir]
    unzip_process = subprocess.Popen(unzip_command)
    unzip_process.wait()  # Wait for each unzip operation to complete

time.sleep(5)
!rm -f d1.zip d2.zip d3.zip d4.zip d5.zip d6.zip d7.zip d8.zip d9.zip d10.zip d11.zip d12.zip

# Function to recursively find 'seal' directories and get all image paths inside
def get_all_image_paths_and_labels(directory, use_numeric_labels=True):
    image_paths = []
    labels = []
    label_to_class = {}

    base_path = pathlib.Path(directory)
    class_names = []

    # First pass: gather all class names from all 'crops/seal' dirs
    for seal_dir in base_path.glob('**/crops/seal'):
        parts = seal_dir.parts
        if "Unk" in parts:
            print("loop1", parts)
            class_index = parts.index("Unk") + 1
        else:
            class_index = parts.index("seal") - 3

        class_name = parts[class_index]
        class_names.append(class_name)

    class_names = sorted(set(class_names))

    # Map classes to labels
    if use_numeric_labels:
        class_indices = {name: idx for idx, name in enumerate(class_names)}
        label_to_class = {idx: name for name, idx in class_indices.items()}
    else:
        class_indices = {name: name for name in class_names}

    # Second pass: collect image paths and labels
    for seal_dir in base_path.glob('**/crops/seal'):
        parts = seal_dir.parts

        if "Unk" in parts:
            print("loop2", parts)
            class_index = parts.index("Unk") + 1
        else:
            class_index = parts.index("seal") - 3

        class_name = parts[class_index]
        label = class_indices[class_name]

        class_image_paths = list(seal_dir.glob('*.jpg')) + \
                            list(seal_dir.glob('*.jpeg')) + \
                            list(seal_dir.glob('*.JPG')) + \
                            list(seal_dir.glob('*.png'))

        image_paths.extend([str(p) for p in class_image_paths])
        labels.extend([label] * len(class_image_paths))

    return image_paths, labels, label_to_class

# --- Step 3: Use the function ---
image_paths, labels, label_to_class = get_all_image_paths_and_labels(source_dir, use_numeric_labels=True)

# --- Step 4: Print results ---
print(f"Total number of seal images: {len(image_paths)}")
for i, path in enumerate(image_paths[:10]):
    print(f"Image {i+1}: {path} | Label: {labels[i]}")



Total number of seal images: 8488
Image 1: data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0749.jpg | Label: 549
Image 2: data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0754.jpg | Label: 549
Image 3: data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0778.jpg | Label: 549
Image 4: data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0782.jpg | Label: 549
Image 5: data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0746.jpg | Label: 549
Image 6: data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0775.jpg | Label: 549
Image 7: dat

In [ ]:
# View the labels and label_to_class

print(f"Labels: {labels[:10]}") # Print first 10 labels
print(f"Label to Class mapping: {label_to_class}")


Labels: [549, 549, 549, 549, 549, 549, 549, 549, 549, 549]
Label to Class mapping: {0: '000 RedWhite', 1: '003 RedWhite', 2: '100 Orange', 3: '102 Orange', 4: '103 Orange', 5: '104 Orange', 6: '106 Orange', 7: '107 Orange', 8: '108 Orange', 9: '11 Orange', 10: '110 Orange', 11: '111 Orange', 12: '113 Orange', 13: '114 Orange', 14: '115 Orange', 15: '116 Orange', 16: '117 Orange', 17: '118 Orange', 18: '119 Orange', 19: '12 Orange', 20: '120 Orange', 21: '121 Orange', 22: '122 Orange', 23: '123 Orange', 24: '124 Orange', 25: '125 Orange', 26: '126 Orange', 27: '127 Orange', 28: '128 Orange', 29: '129 Orange', 30: '13 Orange', 31: '131 Orange', 32: '132 Orange', 33: '133 Orange', 34: '134 Orange', 35: '136 Orange', 36: '139 Orange', 37: '146 Orange', 38: '147 Orange', 39: '148 Orange', 40: '149 Orange', 41: '150 Orange', 42: '152 Orange', 43: '153 Orange', 44: '154 Orange', 45: '155 Orange', 46: '156 Orange', 47: '157 Orange', 48: '158 Orange', 49: '159 Orange', 50: '16 Orange', 51: '160

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

def ensure_min_samples_per_class(image_paths, image_labels, min_samples=2, test_size=0.3, random_state=None):
    """
    Ensure all classes in image_labels have at least min_samples samples in the temp split.

    Returns:
        train_image_paths, temp_image_paths, train_labels, temp_labels
    """
    image_paths = np.array(image_paths)
    image_labels = np.array(image_labels)

    while True:
        train_paths, temp_paths, train_labels, temp_labels = train_test_split(
            image_paths, image_labels,
            test_size=test_size,
            stratify=image_labels,
            random_state=random_state
        )

        _, temp_counts = np.unique(temp_labels, return_counts=True)

        if np.all(temp_counts >= min_samples):
            return list(train_paths), list(temp_paths), list(train_labels), list(temp_labels)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# Directory containing your images
image_dir = '/content/data/'

# Load image paths and labels
image_paths, image_labels, labels_to_classes = get_all_image_paths_and_labels(image_dir, '_cs_')

# Show examples
for img, lbl in zip(image_paths[:10], image_labels[:10]):
    print(f"Image: {img}, Label: {lbl}")

# Convert to NumPy arrays
image_paths = np.array(image_paths)
image_labels = np.array(image_labels)

# Identify classes with fewer than 8 samples
unique_labels, label_counts = np.unique(image_labels, return_counts=True)
few_sample_classes = unique_labels[label_counts < 8]

# Mask for few and many sample classes
few_sample_mask = np.isin(image_labels, few_sample_classes)
many_sample_mask = ~few_sample_mask

# Step 1: Split only the classes with sufficient samples
train_image_paths, temp_image_paths, train_labels, temp_labels = ensure_min_samples_per_class(
    image_paths[many_sample_mask],
    image_labels[many_sample_mask],
    min_samples=2
)

# Step 2: Split the temp set into validation and test
val_image_paths, test_image_paths, val_labels, test_labels = train_test_split(
    temp_image_paths,
    temp_labels,
    test_size=0.5,
    random_state=42,
    stratify=temp_labels
)

# Step 3: Add the few-sample-class images to the training set
train_image_paths = np.concatenate([train_image_paths, image_paths[few_sample_mask]])
train_labels = np.concatenate([train_labels, image_labels[few_sample_mask]])

# Final split sizes
print(f"Training set size: {len(train_image_paths)}")
print(f"Validation set size: {len(val_image_paths)}")
print(f"Test set size: {len(test_image_paths)}")


Image: /content/data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0749.jpg, Label: 549
Image: /content/data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0754.jpg, Label: 549
Image: /content/data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0778.jpg, Label: 549
Image: /content/data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0782.jpg, Label: 549
Image: /content/data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0746.jpg, Label: 549
Image: /content/data/yolo9_tf_24_07_16/TF10_Male (left and right chin right flipper scar on the lips)/Arredondo_2021-11-11_Parry/crops/seal/DJI_0775.jpg, Label: 549
Image: /co

In [ ]:
# Print some examples to verify
print("=== Train Samples ===")
for img, lbl in zip(train_image_paths[:5], train_labels[:5]):
    print(f"Train Image: {img}, Label: {lbl}")

print("\n=== Validation Samples ===")
for img, lbl in zip(val_image_paths[:5], val_labels[:5]):
    print(f"Validation Image: {img}, Label: {lbl}")

print("\n=== Test Samples ===")
for img, lbl in zip(test_image_paths[:5], test_labels[:5]):
    print(f"Test Image: {img}, Label: {lbl}")


=== Train Samples ===
Train Image: /content/data/yolo9_cs_24_07_16/377 Yellow/unk_2016-12-30_SAN/crops/seal/12.30.2016.SAN3.jpg, Label: 67
Train Image: /content/data/yolo9_cs_24_07_16/155 Orange/unk_2019-02-22_RocasYam (AFC)/crops/seal/DSC04334.jpg, Label: 45
Train Image: /content/data/yolo9_hw_24_07_16/53286_unk/DougCheeseman_2009-01-17_Antarctica/crops/seal/2009-01-17_Doug_Cheeseman_-__12076.jpg, Label: 97
Train Image: /content/data/yolo9_tf_24_07_16/TF06_Fem (neck pattern and distinctive right flipper patt big scar right flipper)/Martinez_2018-02-20_Parry/crops/seal/20-02-18 (9).jpg, Label: 545
Train Image: /content/data/yolo9_cs_24_07_16/9 Orange/unk_2016-01-29_unk/crops/seal/DSCN4885.jpg, Label: 522

=== Validation Samples ===
Validation Image: /content/data/yolo9_sr_24_07_16/LSR06_unk/unk_2019-01-25_unk/crops/seal/DSC_01882.jpg, Label: 535
Validation Image: /content/data/yolo9_hw_24_07_16/59900_Male/EsthervanderStappen_2017-12-12_Antarctica/crops/seal/2017-12-12_Esther_van_der_St

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torchvision import models
from torch.utils.data import DataLoader
from byol_pytorch import BYOL
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import os
# from torchvision import datasets


In [ ]:
import pathlib
from collections import defaultdict
from sklearn.model_selection import train_test_split
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from joblib import Parallel, delayed
from tqdm import tqdm
import numpy as np

# ====== Constants ======
batch_size = 32
image_size = 224
source_dir = "data"  # Define the source directory where your images are stored

# ====== Image validation ======
def is_valid_image(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()
        return True
    except Exception:
        print(f"Invalid or corrupt image file: {image_path}")
        return False

def filter_valid_images_and_labels(image_paths, labels):
    results = Parallel(n_jobs=4)(delayed(is_valid_image)(p) for p in tqdm(image_paths, desc="Validating Images"))
    valid_paths = [p for p, valid in zip(image_paths, results) if valid]
    valid_labels = [l for l, valid in zip(labels, results) if valid]
    return valid_paths, valid_labels

# ====== Transforms ======
byol_augment = transforms.Compose([
    transforms.RandomResizedCrop(image_size, scale=(0.2, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# ====== Dataset Class ======
class SealDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        if self.labels:
            return img, self.labels[idx]
        return img

# ====== Prepare data ======
# Assuming you have already extracted the image_paths and string labels
# Adjust the unpacking to capture the third return value (label_to_class)
seal_image_paths, seal_labels, label_to_class = get_all_image_paths_and_labels(source_dir, use_numeric_labels=True)
seal_image_paths, seal_labels = filter_valid_images_and_labels(seal_image_paths, seal_labels)


# Convert string labels to numeric labels (if needed)
unique_labels = sorted(set(seal_labels))
label_to_class = {label: idx for idx, label in enumerate(unique_labels)}  # You can skip this if you already have numeric labels
seal_labels = [label_to_class[label] for label in seal_labels]  # Convert to numeric labels

# Split data into training and testing sets (if not already done)
train_paths, test_paths, train_labels, test_labels = train_test_split(
    seal_image_paths, seal_labels, test_size=0.2, random_state=42
)

# Split data into train, validation, and test sets (if not already done)
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=0.2, random_state=42
)

# ====== Datasets & Dataloaders ======
train_dataset = SealDataset(train_paths, train_labels, transform=byol_augment)
test_dataset = SealDataset(test_paths, test_labels, transform=byol_augment)
val_dataset = SealDataset(val_paths, val_labels, transform=byol_augment)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# ====== Logging ======
print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")
print(f"Testing images: {len(test_dataset)}")

# Preview a few examples
for img, lbl in zip(train_paths[:5], train_labels[:5]):
    print(f"Train: {img}, Label: {lbl}")


Validating Images: 100%|██████████| 8488/8488 [00:10<00:00, 778.94it/s]


Training images: 5432
Validation images: 1358
Testing images: 1698
Train: data/yolo9_cs_24_07_16/134 Orange/unk_2018-01-05_Alcazar (SMW)/crops/seal/P1010267.jpg, Label: 34
Train: data/yolo9_cs_24_07_16/156 Orange/unk_2019-04-30_unk/crops/seal/DSC_0052.jpg, Label: 46
Train: data/yolo9_hw_24_07_16/53335_unk/BarryMiller_2015-01-03_Antarctica/crops/seal/2015-01-03_Barry_Miller_439372.jpg, Label: 130
Train: data/yolo9_sr_24_07_16/LSR06_unk/unk_2019-07-06_unk/crops/seal/DSC_8662.jpg, Label: 535
Train: data/yolo9_cs_24_07_16/128 Orange/unk_2017-08-07_unk/crops/seal/IMG_65982.jpg, Label: 28


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
import torch
from torch.utils.data import DataLoader
import torchvision.models as models
from torch import nn

# Set device (CUDA if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained ResNet50 backbone
resnet = models.resnet50(pretrained=True)

# Remove the fully connected layer (fc) and the projection head
resnet.fc = torch.nn.Identity()

# Define a hook function to capture the 2048-dimensional features
features = None
def hook_fn(module, input, output):
    global features
    features = output.view(output.size(0), -1)  # Flatten to (batch_size, 2048)

# Register the hook on the avgpool layer
hook = resnet.avgpool.register_forward_hook(hook_fn)

# Initialize BYOL with the modified ResNet50
learner = BYOL(
    resnet,
    image_size=224,
    hidden_layer="avgpool"  # Ensures we hook into this layer
)

# Move the model to the correct device (CUDA or CPU)
learner.to(device)

# Set the model in training mode
learner.train()




/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 135MB/s]


BYOL(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Co

In [ ]:
# Initialize the DataLoader for training data
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Set the optimizer (Adam)
optimizer = torch.optim.Adam(learner.parameters(), lr=3e-4)

# Training loop
epochs = 10
for epoch in range(epochs):
    learner.train()
    for images, labels in train_loader:
        images = images.to(device)  # Move images to device

        # Pass the images through BYOL, which handles augmentations internally
        loss = learner(images)

        # Backpropagate and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Update the moving average of the target encoder
    learner.update_moving_average()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# Save the trained model
torch.save(learner.state_dict(), 'byol_model.pth')

Epoch [1/10], Loss: 1.5816
Epoch [2/10], Loss: 1.5333
Epoch [3/10], Loss: 1.4707
Epoch [4/10], Loss: 1.5573
Epoch [5/10], Loss: 1.3471
Epoch [6/10], Loss: 1.7002
Epoch [7/10], Loss: 1.3120
Epoch [8/10], Loss: 1.3341
Epoch [9/10], Loss: 1.2286
Epoch [10/10], Loss: 1.1896


In [ ]:
def get_embeddings(data_loader, learner, device):
    learner.eval()  # Set the model to evaluation mode
    embeddings = []
    labels_list = []

    with torch.no_grad():  # Disable gradient calculation
        for images, labels in tqdm(data_loader, desc="Extracting Embeddings"):
            images = images.to(device)  # Move images to device
            labels_list.append(labels)

            # Forward pass to capture the embeddings from the hook
            learner(images)

            # `features` now has 2x images. Keep only the first half.
            batch_size = images.size(0)
            embeddings.append(features.cpu().numpy()[:batch_size, :])

    embeddings = np.concatenate(embeddings, axis=0)
    labels_list = np.concatenate(labels_list, axis=0)
    return embeddings, labels_list

In [ ]:
# Extract embeddings for the training set
train_embeddings, train_labels = get_embeddings(train_loader, learner, device)

# Extract embeddings for the validation set
val_embeddings, val_labels = get_embeddings(val_loader, learner, device)

# Check the shape of the embeddings
print(f"Train embeddings shape: {train_embeddings.shape}")
print(f"Validation embeddings shape: {val_embeddings.shape}")


Extracting Embeddings: 100%|██████████| 43/43 [01:20<00:00,  1.88s/it]

Train embeddings shape: (5432, 2048)
Validation embeddings shape: (1358, 2048)


In [ ]:
# Convert the NumPy arrays to PyTorch tensors
train_embeddings_tensor = torch.tensor(train_embeddings, dtype=torch.float32)
val_embeddings_tensor = torch.tensor(val_embeddings, dtype=torch.float32)

train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)
val_labels_tensor = torch.tensor(val_labels, dtype=torch.long)

# Print the shapes of the tensors to verify the conversion
print(f"Train embeddings tensor shape: {train_embeddings_tensor.shape}")
print(f"Validation embeddings tensor shape: {val_embeddings_tensor.shape}")
print(f"Train labels tensor shape: {train_labels_tensor.shape}")
print(f"Validation labels tensor shape: {val_labels_tensor.shape}")



Train embeddings tensor shape: torch.Size([5432, 2048])
Validation embeddings tensor shape: torch.Size([1358, 2048])
Train labels tensor shape: torch.Size([5432])
Validation labels tensor shape: torch.Size([1358])


In [ ]:
print("Example values of train_embeddings:")
print(train_embeddings_tensor[:5])  # Print the first 5 examples
print("\nExample values of val_embeddings:")
print(val_embeddings_tensor[:5])  # Print the first 5 examples
print("\nExample values of train_labels:")
print(train_labels_tensor[:5])  # Print the first 5 examples
print("\nExample values of val_labels:")
print(val_labels_tensor[:5])  # Print the first 5 examples


Example values of train_embeddings:
tensor([[0.2513, 0.6940, 0.2678,  ..., 0.0304, 0.0561, 0.1001],
        [0.4667, 0.1399, 0.0936,  ..., 0.0088, 0.3816, 0.3609],
        [1.5078, 0.0609, 0.1816,  ..., 0.0386, 0.1289, 0.2806],
        [0.9525, 0.2678, 0.0938,  ..., 0.0363, 0.1030, 0.1910],
        [0.9732, 0.1578, 0.3186,  ..., 0.0963, 0.8921, 0.2119]])

Example values of val_embeddings:
tensor([[0.0323, 1.2673, 0.2325,  ..., 0.6188, 0.5888, 0.4084],
        [0.2025, 0.3705, 0.3092,  ..., 0.1201, 0.2265, 0.3039],
        [0.0665, 0.9227, 0.4037,  ..., 0.1761, 0.2269, 0.2424],
        [0.1952, 0.1752, 0.2065,  ..., 0.3249, 0.6541, 0.2826],
        [0.2733, 0.4548, 0.3472,  ..., 0.6040, 0.3913, 0.2608]])

Example values of train_labels:
tensor([486, 339,  28, 547, 448])

Example values of val_labels:
tensor([298, 485,  73,  20, 533])


In [ ]:
database_embeddings = torch.cat([train_embeddings_tensor, val_embeddings_tensor])
database_labels = torch.cat([train_labels_tensor, val_labels_tensor])


In [ ]:
# Convert NumPy arrays to PyTorch tensors (explicitly using .cpu() to avoid GPU-TensorFlow weirdness)
train_embeddings_tensor = torch.from_numpy(train_embeddings).float().cpu()
val_embeddings_tensor = torch.from_numpy(val_embeddings).float().cpu()

train_labels_tensor = torch.from_numpy(train_labels).long().cpu()
val_labels_tensor = torch.from_numpy(val_labels).long().cpu()

# Concatenate them to form the database
database_embeddings = torch.cat([train_embeddings_tensor, val_embeddings_tensor], dim=0)
database_labels = torch.cat([train_labels_tensor, val_labels_tensor], dim=0)


In [ ]:
print(database_embeddings[:5])
print(database_labels[:5])


tensor([[0.2513, 0.6940, 0.2678,  ..., 0.0304, 0.0561, 0.1001],
        [0.4667, 0.1399, 0.0936,  ..., 0.0088, 0.3816, 0.3609],
        [1.5078, 0.0609, 0.1816,  ..., 0.0386, 0.1289, 0.2806],
        [0.9525, 0.2678, 0.0938,  ..., 0.0363, 0.1030, 0.1910],
        [0.9732, 0.1578, 0.3186,  ..., 0.0963, 0.8921, 0.2119]])
tensor([486, 339,  28, 547, 448])


In [ ]:
def find_top_k_matches(query_embedding, database_embeddings, k=10):
    # Compute cosine similarities
    query_embedding = query_embedding.to(database_embeddings.device)  # Ensure query_embedding is on the same device
    similarities = torch.nn.functional.cosine_similarity(query_embedding.unsqueeze(0), database_embeddings)
    top_k_indices = similarities.topk(k=k).indices
    return top_k_indices


In [ ]:
from sklearn.metrics import f1_score, balanced_accuracy_score

def evaluate_similarity_search(learner, database_embeddings, database_labels, dataloader, device, k=10):
    correct_top_1 = 0
    correct_top_5 = 0
    correct_top_10 = 0
    total = 0

    all_true_labels = []
    all_pred_labels = []

    # Move database_embeddings and database_labels to the correct device
    # No need to convert to numpy and back, just move to the correct device
    database_embeddings = database_embeddings.to(device)
    database_labels = database_labels.to(device)

    learner.eval()  # Set the model to evaluation mode

    with torch.no_grad():  # Disable gradient computation
        for images, labels in dataloader:
            images = images.to(device)  # Move images to device
            labels = labels.to(device)  # Move labels to device

            # Extract query embeddings using your get_embeddings function
            learner(images)
            query_embeddings = features.cpu()

            # Loop through each query in the batch
            for i in range(len(labels)):
                query_embedding = torch.tensor(query_embeddings[i]).to(device)  # Ensure the query embedding is on the device

                # Find top-k matches in the database
                top_k_indices = find_top_k_matches(query_embedding, database_embeddings, k=k)

                # Retrieve top-k labels for the found indices
                top_k_labels = database_labels[top_k_indices]

                # Append true and predicted labels for accuracy calculation
                all_true_labels.append(labels[i].cpu().item())  # Move to CPU for metric calculation
                all_pred_labels.append(top_k_labels[0].cpu().item())  # Take top-1 prediction

                # Update correct counts for top-1, top-5, and top-10
                if labels[i].cpu() in top_k_labels[:1].cpu():  # Check if true label is in top-1
                    correct_top_1 += 1
                if labels[i].cpu() in top_k_labels[:5].cpu():  # Check if true label is in top-5
                    correct_top_5 += 1
                if labels[i].cpu() in top_k_labels[:10].cpu():  # Check if true label is in top-10
                    correct_top_10 += 1
                total += 1

    # Calculate accuracy metrics
    top_1_accuracy = correct_top_1 / total
    top_5_accuracy = correct_top_5 / total
    top_10_accuracy = correct_top_10 / total

    # Calculate F1 score
    f1 = f1_score(all_true_labels, all_pred_labels, average='weighted')

    # Calculate Balanced Accuracy
    balanced_acc = balanced_accuracy_score(all_true_labels, all_pred_labels)

    return top_1_accuracy, top_5_accuracy, top_10_accuracy, f1, balanced_acc


In [ ]:
def evaluate_similarity_search(learner, database_embeddings, database_labels, dataloader, k=10):
    correct_top_1 = 0
    correct_top_5 = 0
    correct_top_10 = 0
    total = 0

    all_true_labels = []
    all_pred_labels = []

    database_embeddings = database_embeddings.to(device)  # Ensure database_embeddings is on the same device
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            # Forward pass to capture the embeddings from the hook
            learner(images)
            cls_token_output = features.cpu()

            for i in range(len(labels)):
                query_embedding = cls_token_output[i]
                print(cls_token_output.shape, database_embeddings.shape, query_embedding.shape)

                top_k_indices = find_top_k_matches(query_embedding, database_embeddings, k=k)
                top_k_indices = top_k_indices.cpu()  # Move indices to CPU
                top_k_labels = database_labels[top_k_indices].cpu()  # Ensure labels are on CPU

                all_true_labels.append(labels[i].cpu().item())
                all_pred_labels.append(top_k_labels[0].item())  # Take the top-1 prediction

                if labels[i].cpu() in top_k_labels[:1]:  # Move labels[i] to CPU
                    correct_top_1 += 1
                if labels[i].cpu() in top_k_labels[:5]:  # Move labels[i] to CPU
                    correct_top_5 += 1
                if labels[i].cpu() in top_k_labels[:10]:  # Move labels[i] to CPU
                    correct_top_10 += 1
                total += 1

    top_1_accuracy = correct_top_1 / total
    top_5_accuracy = correct_top_5 / total
    top_10_accuracy = correct_top_10 / total

    # Calculate F1 score
    f1 = f1_score(all_true_labels, all_pred_labels, average='weighted')

    # Calculate Balanced Accuracy
    balanced_acc = balanced_accuracy_score(all_true_labels, all_pred_labels)

    return top_1_accuracy, top_5_accuracy, top_10_accuracy, f1, balanced_acc

In [ ]:
top_1_acc, top_5_acc, top_10_acc, f1, balanced_acc = evaluate_similarity_search(learner, database_embeddings, database_labels, test_loader)
print(f"\nTop-1 Accuracy: {top_1_acc} \nTop-5 Accuracy: {top_5_acc} \nTop-10 Accuracy: {top_10_acc} \nF1 Score: {f1} \nBalanced Accuracy: {balanced_acc}")

torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([2048])
torch.Size([64, 2048]) torch.Size([6790, 2048]) torch.Size([20

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:2524: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")
